In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import DBSCAN
import re
pd.set_option('display.max_columns', None)

In [ ]:
#list of all MRE types
MREs_list = ["TGCGCAC", "TGCGCGC", "TGCGCCC", "TGCGCTC", "TGCACAC", "TGCACGC", "TGCACCC", "TGCACTC", 
             "GAGTGCA", "GGGTGCA", "GCGTGCA", "GTGTGCA", "GAGCGCA", "GGGCGCA", "GCGCGCA", "GTGCGCA"]

#initiate final DataFrame
overlapping_ranges_all = pd.DataFrame(columns=["df1_row", "df2_row", "Cluster", "MRE_start","MRE_type","MRE_zone_start","MRE_zone_end","MRE_type_str","TrName","Chr","Source","Feature","Start","End","Score","Strand","Frame","Attribute"])

#loop through chromosome FASTAs to discover Metal Responsive Element clusters near genes
#filter by MRE cluster types found in the Metallothionein promoter
for i in range(1,19):
    MRECW_df = pd.DataFrame(columns=["MRE_start", "MRE_type", "Chromosome"])
    
    #import gtf of gene predictions for the respective chromosome
    df_chr1_gtf = pd.read_csv(f"./chr_gtf/chr{i}.gtf", delimiter="\t", header=None)
    df_chr1_names = pd.read_csv(f"./chr_gtf/uniq_CDS{i}.txt", delimiter="\t", header=None)

    df_chr1_gtf.columns=['Chr', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand', 'Frame', 'Attribute']
    df_chr1_gtf.insert(0, "TrName", df_chr1_names)
    
    #import chromosome FASTA for the respective chromosme
    chr17 = pd.read_csv(f"./chr_fasta/chr{i}.fa", names=["seq", "score"])
    chr17_df = pd.DataFrame([*chr17["seq"][1]], columns=["Nucleotide"])
    chr17_df["Coordinate"] = chr17_df.index
    chr17_df["Score"] = 1
    chr17_df["Coordinate"] = chr17_df["Coordinate"] + 1
    
    MRE_df = pd.DataFrame(columns=["MRE_start", "MRE_type"])
    
    #find all MRE sites in the chromosome
    for MRE in MREs_list:
        all_MREs = [m.start() for m in re.finditer(MRE, chr17["seq"][1])]
        MRE_df_temp = pd.DataFrame(all_MREs, columns=["MRE_start"])
        MRE_df_temp["MRE_type"] = MRE
        MRE_df = pd.concat([MRE_df, MRE_df_temp], axis=0)
    
    #caclulate MRE counts
    MRE_freq = []
    MRE_frequencies = pd.DataFrame()
    for MRE in MREs_list:
        MRE_freq.append(MRE_df[MRE_df["MRE_type"]==MRE].shape[0])
    MRE_frequencies["MRE_type"] = MREs_list
    MRE_frequencies["Occurrences"] = MRE_freq

    MRE_df_reset_idx = MRE_df.sort_values(by="MRE_start").reset_index(drop=True)
    
    #cluster MREs within 350bp of one another 
    db = DBSCAN(eps=350, min_samples=2).fit(MRE_df_reset_idx["MRE_start"].values.reshape(-1,1))
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    
    #filter clusters to have exactly 3 MREs only
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    MRE_df_reset_idx["Cluster"] = labels
    clustered_MREs_df = MRE_df_reset_idx[MRE_df_reset_idx["Cluster"] != -1]
    clustered_MREs_df.value_counts(subset="Cluster")
    counts = clustered_MREs_df["Cluster"].value_counts(dropna=False)
    clustered_MREs_df_over2 = clustered_MREs_df[clustered_MREs_df["Cluster"].map(counts)>2]
    clustered_MREs_df_over2_below4 = clustered_MREs_df_over2[clustered_MREs_df_over2["Cluster"].map(counts)<4]
    
    #sort data structures
    df1 = clustered_MREs_df_over2_below4.groupby(["Cluster"])["MRE_type"].apply(list).reset_index()
    clustered_MREs_df_over2_below4 = clustered_MREs_df_over2_below4.groupby(["Cluster"])["MRE_start"].apply(list).reset_index()
    clustered_MREs_df_over2_below4["MRE_type"] = df1["MRE_type"]
    clustered_MREs_df_over2_below4["MRE_type_str"] = clustered_MREs_df_over2_below4["MRE_type"].astype(str)
    clustered_MREs_df_over2_below4["MRE_zone_start"] = clustered_MREs_df_over2_below4["MRE_start"].apply(min)
    clustered_MREs_df_over2_below4["MRE_zone_end"] = clustered_MREs_df_over2_below4["MRE_start"].apply(max)
    
    #filter MREs to be identical to the ones found in the Metallothionein promoter
    #chr1:
    df1 = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['GAGCGCA', 'TGCACGC', 'GTGTGCA']"]
    df1_rev = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['GTGTGCA', 'TGCACGC', 'GAGCGCA']"]
    df2_rc = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['TGCGCTC', 'GCGTGCA', 'TGCACAC']"]
    df2_rc_rev = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['TGCACAC', 'GCGTGCA', 'TGCGCTC']"]
    #chr4:
    df3 = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['GCGTGCA', 'TGCACAC', 'GCGTGCA']"]
    df3_rev = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['GCGTGCA', 'TGCACAC', 'GCGTGCA']"]
    df4_rc = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['TGCACGC', 'GTGTGCA', 'TGCACGC']"]
    df4_rc_rev = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['TGCACGC', 'GTGTGCA', 'TGCACGC']"]
    #chr1_CW_variant
    df5 = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['GAGTGCA', 'TGCACGC', 'GTGTGCA']"]
    df5_rev = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['GTGTGCA', 'TGCACGC', 'GAGTGCA']"]
    df6_rc = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['TGCACTC', 'GCGTGCA', 'TGCACAC']"]
    df6_rc_rev = clustered_MREs_df_over2_below4[clustered_MREs_df_over2_below4["MRE_type_str"]=="['TGCACAC', 'GCGTGCA', 'TGCACTC']"]
    
    frames = [df1, df1_rev, df2_rc, df2_rc_rev, df3, df3_rev, df4_rc, df4_rc_rev, df5, df5_rev, df6_rc, df6_rc_rev]
    result = pd.concat(frames)

    #prepare dataframes for gene discovery
    df1 = result.sort_values(by='MRE_zone_start').reset_index(drop=True)
    df2 = df_chr1_gtf.sort_values(by='Start').reset_index(drop=True)
    
    #discover genes within 150 bp of MRE clusters
    df2["Start"] = df2["Start"] - 150
    df2["End"] = df2["End"] + 150
    
    #create a boolean mask for overlapping ranges of MRE clusters and predicted genes
    mask = (df1['MRE_zone_end'].values[:, np.newaxis] >= df2['Start'].values) & (df1['MRE_zone_start'].values[:, np.newaxis] <= df2['End'].values)

    # Check if there is any overlap (True in the mask) - discover genes
    if mask.any():
        overlapping_rows = np.where(mask)
        for i, j in zip(*overlapping_rows):
            print(f"Overlap between row {i} in df1 and row {j} in df2.")

    #finalise data structures
    overlap_indices = np.argwhere(mask)
    overlapping_ranges = pd.DataFrame(overlap_indices, columns=['df1_row', 'df2_row'])
    overlapping_ranges = overlapping_ranges.join(df1, on='df1_row')
    overlapping_ranges = overlapping_ranges.join(df2, on='df2_row')
    overlapping_ranges.drop_duplicates(subset="TrName")
    overlapping_ranges["Chromosome"] = i
    overlapping_ranges_all = pd.concat([overlapping_ranges_all, overlapping_ranges])
    print(i, "/18")

In [ ]:
#import UPIMAPI annotations
upimapi_df = pd.read_csv("/home/Maxim/Bioinformatics/General_software/UPIMAPI/output/CambridgeChrs_adjusted/CambridgeChrs_adjusted/merged/Cambridge_UPIMAPI.tsv", delimiter="\t")
qseqid_df = upimapi_df["qseqid"].copy()
upimapi_df.insert(0, "TrName", qseqid_df.str[:-2])
test_df_upimapi = upimapi_df.astype(str).groupby("TrName")[['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen',
       'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'Entry',
       'Entry Name', 'Gene Names', 'Protein names', 'EC number',
       'Function [CC]', 'Pathway', 'Keywords', 'Protein existence',
       'Gene Ontology (GO)', 'Protein families', 'Taxonomic lineage',
       'Taxonomic lineage (Ids)', 'Organism (ID)', 'BioCyc', 'BRENDA', 'CDD',
       'eggNOG', 'Ensembl', 'InterPro', 'KEGG', 'Pfam', 'Reactome', 'RefSeq',
       'UniPathway', 'Taxonomic lineage (SUPERKINGDOM)',
       'Taxonomic lineage (PHYLUM)', 'Taxonomic lineage (CLASS)',
       'Taxonomic lineage (ORDER)', 'Taxonomic lineage (FAMILY)',
       'Taxonomic lineage (GENUS)', 'Taxonomic lineage (SPECIES)']].agg('! '.join).apply(lambda x: x[:].str.split('!')).reset_index()

In [ ]:
#couple discovered candidate genes to their annotations
final = overlapping_ranges_all.merge(test_df_upimapi, on=["TrName"], how="left")

#gene/MRE cluster coordinates columns
final["MRE_zone_start_str"] = final["MRE_zone_start"].astype(str)
final["MRE_zone_end_str"] = final["MRE_zone_end"].astype(str)
final["MRE_coordinates"] = final["MRE_zone_start_str"] + " - " + final["MRE_zone_end_str"]

final["Start_str"] = final["Start"].astype(str)
final["End_str"] = final["End"].astype(str)
final["Transcript_coordinates"] = final["Start_str"] + " - " + final["End_str"]

#filter redundant columns and save the list of candidantes
final_short = final[["Chr","MRE_type", "MRE_coordinates", "TrName", "Transcript_coordinates", "Entry"]]
final_short.rename(columns={"TrName":"Transcript_name", "Entry":"UniProt_Entry", "Chr":"Chromosome"}).to_csv("./MRE_gene_candidates.csv")